# 🚴 Processamento de Dados - 2. Limpeza e Criação de Campos Analíticos

Este notebook realiza a limpeza de dados e a engenharia de recursos (feature engineering), conforme detalhado na Seção 4 do `README.md`.

## 1. Carregamento do Checkpoint Unificado

Carregamos o arquivo gerado pelo notebook `01_data_unification_and_save.ipynb` para iniciar a fase de tratamento.

---

In [1]:
import pandas as pd
import numpy as np
import os
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# ----------------------------------------------------------------------
# 1. CONFIGURAÇÃO DE CAMINHOS E CARREGAMENTO
# ----------------------------------------------------------------------

# ⚠️ AJUSTE ESTE CAMINHO para o local da sua pasta no Drive!
# Deve ser o mesmo caminho usado no script de unificação (01).
BASE_PATH = '/content/drive/MyDrive/Coursera_google_analytics/'

# Caminho do arquivo unificado gerado no script anterior
UNIFIED_FILE_PATH = os.path.join(BASE_PATH, 'cyclistic_viagens_unificadas_bruto.csv')

print(f"Carregando arquivo unificado bruto: '{UNIFIED_FILE_PATH}'")

# Carregar o arquivo unificado
# O low_memory=False ajuda a garantir que o Pandas identifique corretamente os tipos de dados
df = pd.read_csv(UNIFIED_FILE_PATH, low_memory=False)

print(f"✅ DataFrame 'df' carregado com sucesso. Total de linhas: {len(df)}")
print("Iniciando a limpeza de dados e criação de campos analíticos...")


Carregando arquivo unificado bruto: '/content/drive/MyDrive/Coursera_google_analytics/cyclistic_viagens_unificadas_bruto.csv'
✅ DataFrame 'df' carregado com sucesso. Total de linhas: 5055832
Iniciando a limpeza de dados e criação de campos analíticos...


In [4]:
df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,7569BC890583FCD7,classic_bike,2025-01-21 17:23:54.538,2025-01-21 17:37:52.015,Wacker Dr & Washington St,KA1503000072,McClurg Ct & Ohio St,TA1306000029,41.883143,-87.637242,41.892592,-87.617289,member
1,013609308856B7FC,electric_bike,2025-01-11 15:44:06.795,2025-01-11 15:49:11.139,Halsted St & Wrightwood Ave,TA1309000061,Racine Ave & Belmont Ave,TA1308000019,41.929147,-87.649153,41.939743,-87.658865,member
2,EACACD3CE0607C0D,classic_bike,2025-01-02 15:16:27.730,2025-01-02 15:28:03.230,Southport Ave & Waveland Ave,13235,Broadway & Cornelia Ave,13278,41.948226,-87.664071,41.945529,-87.646439,member
3,EAA2485BA64710D3,classic_bike,2025-01-23 08:49:05.814,2025-01-23 08:52:40.047,Southport Ave & Waveland Ave,13235,Southport Ave & Roscoe St,13071,41.948226,-87.664071,41.943739,-87.664020,member
4,7F8BE2471C7F746B,electric_bike,2025-01-16 08:38:32.338,2025-01-16 08:41:06.767,Southport Ave & Waveland Ave,13235,Southport Ave & Roscoe St,13071,41.948226,-87.664071,41.943739,-87.664020,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5055827,D13C1FED7727A9C0,electric_bike,2025-10-11 17:16:21.046,2025-10-11 17:31:02.868,Michigan Ave & Ida B Wells Dr,CHI00250,NaN,NaN,41.876203,-87.624464,41.890000,-87.630000,member
5055828,EC76AA5F4AB2D806,electric_bike,2025-10-15 07:54:02.067,2025-10-15 08:03:08.947,Franklin St & Jackson Blvd,CHI02050,NaN,NaN,41.877708,-87.635321,41.890000,-87.620000,member
5055829,B1FF79BE5CF46C1F,electric_bike,2025-10-25 16:44:17.353,2025-10-25 16:46:57.222,W Armitage Ave & N Sheffield Ave,CHI00836,NaN,NaN,41.917805,-87.653449,41.920000,-87.650000,member
5055830,33AA0D781A9A7CD6,electric_bike,2025-10-08 17:10:12.295,2025-10-08 17:17:00.968,W Armitage Ave & N Sheffield Ave,CHI00836,NaN,NaN,41.917805,-87.653449,41.930000,-87.660000,member


#2. Criar colunas com duração das viagens, dia da semana e mês
A métrica ride_length (duração da viagem) é a mais importante. Ela é calculada pela diferença entre o tempo de fim e o tempo de início, e o resultado é convertido para minutos para facilitar a interpretação na análise.

In [5]:
#2. CRIAÇÃO DE CAMPOS ANALÍTICOS
    # ----------------------------------------------------------------------

print("\nCriando campos analíticos ride_length(min), ride_length(hours), day_of_week e day_of_week_name")

#Conversão de tipos de dados para datetime
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

#Criar 'ride_length' (Duração da Viagem)
# A diferença é calculada em Timedelta e convertida para segundos (TOTAL_SECONDS)
df['ride_length(min)'] = (df['ended_at'] - df['started_at']).dt.total_seconds()
df['ride_length(min)'] = df['ride_length(min)']/60 # converter para minutos

# Criar 'ride_length(hours)' (Duração da Viagem em Horas)
df['ride_length(hours)'] = df['ride_length(min)']/60 # converter para horas

#Criar 'day_of_week' (0=Segunda, 6=Domingo) e 'month'
df['day_of_week'] = df['started_at'].dt.dayofweek
df['month'] = df['started_at'].dt.month

# Adicionar o nome do dia da semana (Monday, Tuesday, etc.)
df['day_of_week_name'] = df['started_at'].dt.day_name()


Criando campos analíticos ride_length(min), ride_length(hours), day_of_week e day_of_week_name


## 3. LIMPEZA E TRATAMENTO DE ANOMALIAS

#Tratamento de Duplicatas
Verificar se existes informação repetida nas linhas.
# Tratamento de Nulos
Remove linhas onde falta informação crítica para a análise.
# Filtragem de Viagens Anômalas

Filtra viagens com menos de 60 segundos (1 minuto), que são consideradas anomalias

Remover viagens excessivamente longas (mais de 24h, indicando roubo ou falha grave)

---



In [6]:
# função df.duplicated().sum() para quantificar a existência de linhas completamente idênticas no DataFrame.
df.duplicated().sum()

np.int64(0)

In [7]:
df.shape

(5055832, 18)

In [8]:
#Verificando dados nulos
df.isnull().sum()

,0
ride_id,0
rideable_type,0
started_at,0
ended_at,0
start_station_name,1081467
start_station_id,1081467
end_station_name,1131364
end_station_id,1131364
start_lat,0
start_lng,0


In [9]:
#Utilizando a função dropna para remover linhas com valores nulos.
df.dropna(inplace=True)

In [10]:
#Verificando se todas as linhas com valores nulos foram apagadas do dataframe.
df.isnull().sum()

,0
ride_id,0
rideable_type,0
started_at,0
ended_at,0
start_station_name,0
start_station_id,0
end_station_name,0
end_station_id,0
start_lat,0
start_lng,0


In [11]:
#Nova quantidade de linhas do dataframe.
df.shape

(3362086, 18)

In [12]:
df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length(min),ride_length(hours),day_of_week,month,day_of_week_name
0,7569BC890583FCD7,classic_bike,2025-01-21 17:23:54.538,2025-01-21 17:37:52.015,Wacker Dr & Washington St,KA1503000072,McClurg Ct & Ohio St,TA1306000029,41.883143,-87.637242,41.892592,-87.617289,member,13.957950,0.232632,1,1,Tuesday
1,013609308856B7FC,electric_bike,2025-01-11 15:44:06.795,2025-01-11 15:49:11.139,Halsted St & Wrightwood Ave,TA1309000061,Racine Ave & Belmont Ave,TA1308000019,41.929147,-87.649153,41.939743,-87.658865,member,5.072400,0.084540,5,1,Saturday
2,EACACD3CE0607C0D,classic_bike,2025-01-02 15:16:27.730,2025-01-02 15:28:03.230,Southport Ave & Waveland Ave,13235,Broadway & Cornelia Ave,13278,41.948226,-87.664071,41.945529,-87.646439,member,11.591667,0.193194,3,1,Thursday
3,EAA2485BA64710D3,classic_bike,2025-01-23 08:49:05.814,2025-01-23 08:52:40.047,Southport Ave & Waveland Ave,13235,Southport Ave & Roscoe St,13071,41.948226,-87.664071,41.943739,-87.664020,member,3.570550,0.059509,3,1,Thursday
4,7F8BE2471C7F746B,electric_bike,2025-01-16 08:38:32.338,2025-01-16 08:41:06.767,Southport Ave & Waveland Ave,13235,Southport Ave & Roscoe St,13071,41.948226,-87.664071,41.943739,-87.664020,member,2.573817,0.042897,3,1,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5055673,42BCF854E1A914FB,electric_bike,2025-10-12 09:16:44.600,2025-10-12 09:41:47.759,Wells St & Randolph St Corral,CHI01540,Morgan Ave & 14th Pl,CHI00261,41.884380,-87.633960,41.862378,-87.651062,casual,25.052650,0.417544,6,10,Sunday
5055732,2B229C03A6E7504A,electric_bike,2025-10-03 22:57:31.391,2025-10-03 23:06:04.191,Leavitt St & North Ave,CHI00428,California Ave & Milwaukee Ave,CHI00307,41.910215,-87.682352,41.922695,-87.697153,casual,8.546667,0.142444,4,10,Friday
5055733,DE398B4400F80271,electric_bike,2025-10-28 22:18:23.796,2025-10-28 22:22:26.202,California Ave & Francis Pl,CHI00512,California Ave & Division St,CHI00340,41.918491,-87.697423,41.903029,-87.697474,casual,4.040100,0.067335,1,10,Tuesday
5055802,A8D24DAE33C65360,classic_bike,2025-10-29 13:40:55.649,2025-10-29 14:22:05.718,Blackstone Ave & Hyde Park Blvd,CHI00520,Cottage Grove Ave & 51st St,CHI00502,41.802562,-87.590368,41.803038,-87.606615,member,41.167817,0.686130,2,10,Wednesday


In [13]:
#Viagens com menos de 60 segundos (1 minuto), que são consideradas anomalias.
df = df[df['ride_length(min)'] > 1]

In [14]:
df.shape

(3333476, 18)

In [15]:
#Remover viagens excessivamente longas (mais de 24h, indicando roubo ou falha grave)
df = df[df['ride_length(hours)'] <= 24]
df.shape

(3333377, 18)

In [16]:
#Filtrar por Tipo de Membro válido (Garantia de Integridade Categórica)
df = df[df['member_casual'].isin(['member', 'casual'])]

# 4. SALVAMENTO DO ARQUIVO LIMPO

---

In [17]:
# Redefinindo o DataFrame limpo final
df_clean = df.copy() # Cria uma cópia limpa.


In [18]:
# Salva o DataFrame limpo em um novo CSV (checkpoint)
CLEAN_FILE_PATH = os.path.join(BASE_PATH, 'cyclistic_dados_limpos_analise.csv')
df_clean.to_csv(CLEAN_FILE_PATH, index=False)
print(f"✅ Salvamento concluído!")

✅ Salvamento concluído!


In [19]:
df_clean.shape

(3333377, 18)